<a href="https://colab.research.google.com/github/trunghq0205/python4ds/blob/main/final_project_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----

# Báo cáo cuối kỳ môn Python cho Khoa học Dữ liệu

test commit